# numpy, cupy, scipy.sparse 疎行列*ベクトルの速度計測

In [6]:
import cupy as xp
import numpy as np
import scipy.sparse as sp

In [2]:
import chainer
from chainer import cuda
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dev = cuda.get_device(0)

In [4]:
dev.compute_capability

'35'

Teslaがちゃんと使えるっぽい

## 実際のPDEの大規模疎行列をロード

In [5]:
R = np.load('R.npy')
Dinv = np.load('Dinv.npy')
print R.shape
print Dinv.shape

(40000, 40000)
(40000, 1)


In [7]:
# scipy.sparse
R_sp = sp.csr_matrix(R)
Dinv_sp = sp.csr_matrix(Dinv)

In [17]:
# CUDA
R_xp = cuda.to_gpu(R)
Dinv_xp = cuda.to_gpu(Dinv)

## いざ計測

In [24]:
%%timeit
y_sp = R_sp.dot(Dinv_sp)

1000 loops, best of 3: 1.11 ms per loop


In [28]:
%%timeit
y_ = R.dot(Dinv)

1 loop, best of 3: 269 ms per loop


In [29]:
%%timeit
y_xp = R_xp.dot(Dinv_xp)

100 loops, best of 3: 93.2 ms per loop


## 結論

numpy < cupy << scipy.sparse

※当然大規模疎行列の場合

## appendix

In [88]:
n_row = 2250
n_col = 3

In [89]:
A = np.random.rand(n_row,n_row).astype(np.float64)
x = np.random.rand(n_row, n_col).astype(np.float64)

In [95]:
%%timeit range(10000)
y = A.dot(x)

100 loops, best of 3: 2.28 ms per loop


In [91]:
cuda.get_device(0).use()

In [92]:
A_ = xp.random.rand(n_row,n_row, dtype=xp.float64)
x_ = xp.random.rand(n_row, n_col, dtype=xp.float64)
print A_.device
print A_.shape, x_.shape

<CUDA Device 0>
(2250, 2250) (2250, 3)


In [96]:
%%timeit range(10000)
y_ = A_.dot(x_)

1000 loops, best of 3: 741 µs per loop
